# Car Make Classification using Inception-v4

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/pedermae/Car-make-classification.git

Cloning into 'Car-make-classification'...
remote: Enumerating objects: 102680, done.
remote: Counting objects: 100% (2938/2938), done.
remote: Compressing objects: 100% (2934/2934), done.
remote: Total 102680 (delta 9), reused 18 (delta 4), pack-reused 99742 (from 1)
Receiving objects: 100% (102680/102680), 981.55 MiB | 54.36 MiB/s, done.
Resolving deltas: 100% (6850/6850), done.
Updating files: 100% (33622/33622), done.


In [3]:
%cd /content/Car-make-classification

/content/Car-make-classification


In [4]:
# Import dependencies
import torch
import os
import pandas as pd
import random
import numpy as np
from torchvision.transforms import Compose, ToTensor, RandomAffine, RandomHorizontalFlip, RandomVerticalFlip
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, AvgPool2d, Linear, Dropout, CrossEntropyLoss
from torch.optim import SGD, Adam
from PIL import Image
from tqdm import tqdm


In [5]:
#import sys
#sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/modules')
from dataset import ImageDataset

In [6]:
#Securing reproducability
seed = 0
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [7]:
transforms = Compose([
    ToTensor(), #this converts numpy or Pil image to torch tensor and normalizes it in 0, 1
    RandomAffine((0.05, 0.05)),
    RandomHorizontalFlip(),
    RandomVerticalFlip()
])

## Building Inception-v4

In [8]:
class Conv2d_bn(Module):
    def __init__(self, in_filters, out_filters, kernel_size, strides, padding):
        super().__init__()
        if isinstance(kernel_size, tuple):
            padding_val = (k // 2 for k in kernel_size) if padding == "same" else (0,0)
        else:
            padding_val = kernel_size // 2 if padding == "same" else 0
        self.conv = Conv2d(in_filters, out_filters, kernel_size=kernel_size, stride=strides, padding=padding_val)
        self.bn = BatchNorm2d(out_filters)
        self.relu = ReLU()
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [9]:
class StemBlock(Module):

    def __init__(self):
        super().__init__()
        self.first_block = Sequential(
            Conv2d_bn(in_filters=3, out_filters=32, kernel_size=3, strides=2, padding="valid"),
            Conv2d_bn(in_filters=32, out_filters=32, kernel_size=3, strides=1, padding="valid"),
            Conv2d_bn(in_filters=32, out_filters=64, kernel_size=3, strides=1, padding="same"),
        )
        self.first_left = MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.first_right = Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=2, padding="valid")
        self.second_left =  Sequential(
            Conv2d_bn(in_filters=160, out_filters=64, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding="valid"),
        )
        self.second_right =  Sequential(
            Conv2d_bn(in_filters=160, out_filters=64, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=64, out_filters=64, kernel_size=(7, 1), strides=1, padding="same"),
            Conv2d_bn(in_filters=64, out_filters=64, kernel_size=(1, 7), strides=1, padding="same"),
            Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding="valid"),
        )
        self.third_left = Conv2d_bn(in_filters=192, out_filters=192, kernel_size=3, strides=2, padding="valid")
        self.third_right = MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        x = self.first_block(x)
        x_l = self.first_left(x)
        x_r = self.first_right(x)
        x = torch.cat([x_l, x_r], axis=1)
        x_l = self.second_left(x)
        x_r = self.second_right(x)
        x = torch.cat([x_l, x_r], axis=1)
        x_l = self.third_left(x)
        x_r = self.third_right(x)
        x = torch.cat([x_l, x_r], axis=1)
        return x

In [10]:
class A_block(Module):

    def __init__(self, in_filters):
        super().__init__()
        self.avg_block = Sequential(
            AvgPool2d(kernel_size=3, stride=1, padding=1),
            Conv2d_bn(in_filters=in_filters, out_filters=96, kernel_size=1, strides=1, padding="same"),
        )
        self.one_by_one_block = Conv2d_bn(in_filters=in_filters, out_filters=96, kernel_size=1, strides=1, padding="same")
        self.three_by_three_block =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=64, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding="same"),
        )
        self.five_by_five =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=64, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding="same"),
            Conv2d_bn(in_filters=96, out_filters=96, kernel_size=3, strides=1, padding="same"),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
    def forward(self, x):
        x_1 = self.avg_block(x)
        x_2 = self.one_by_one_block(x)
        x_3 = self.three_by_three_block(x)
        x_4 = self.five_by_five(x)
        x = torch.cat([x_1, x_2, x_3, x_4], axis=1)
        return x

In [11]:
class B_block(Module):

    def __init__(self, in_filters):
        super().__init__()
        self.avg_block = Sequential(
            AvgPool2d(kernel_size=3, stride=1, padding=1),
            Conv2d_bn(in_filters=in_filters, out_filters=128, kernel_size=1, strides=1, padding="same"),
        )
        self.one_by_one_block = Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=1, strides=1, padding="same")

        self.seven_by_seven_block =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=192, out_filters=224, kernel_size=(1, 7), strides=1, padding="same"),
            Conv2d_bn(in_filters=224, out_filters=256, kernel_size=(7, 1), strides=1, padding="same"),
        )

        self.thirteen_by_thirteen_block =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=192, out_filters=192, kernel_size=(1, 7), strides=1, padding="same"),
            Conv2d_bn(in_filters=192, out_filters=224, kernel_size=(7, 1), strides=1, padding="same"),
            Conv2d_bn(in_filters=224, out_filters=224, kernel_size=(1, 7), strides=1, padding="same"),
            Conv2d_bn(in_filters=224, out_filters=256, kernel_size=(7, 1), strides=1, padding="same"),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        x_1 = self.avg_block(x)
        x_2 = self.one_by_one_block(x)
        x_3 = self.seven_by_seven_block(x)
        x_4 = self.thirteen_by_thirteen_block(x)
        x = torch.cat([x_1, x_2, x_3, x_4], axis=1)
        return x

In [12]:
class C_block(Module):

    def __init__(self, in_filters):
        super().__init__()
        self.avg_block = Sequential(
            AvgPool2d(kernel_size=3, stride=1, padding=1),
            Conv2d_bn(in_filters=in_filters, out_filters=256, kernel_size=1, strides=1, padding="same"),
        )
        self.one_by_one_block = Conv2d_bn(in_filters=in_filters, out_filters=256, kernel_size=1, strides=1, padding="same")

        self.branch_a =  Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=1, strides=1, padding="same")
        self.branch_a_left = Conv2d_bn(in_filters=384, out_filters=256, kernel_size=(1, 3), strides=1, padding="same")
        self.branch_a_right = Conv2d_bn(in_filters=384, out_filters=256, kernel_size=(3, 1), strides=1, padding="same")

        self.branch_b =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=384, out_filters=448, kernel_size=(1, 3), strides=1, padding="same"),
            Conv2d_bn(in_filters=448, out_filters=512, kernel_size=(3, 1), strides=1, padding="same"),
        )

        self.branch_b_left = Conv2d_bn(in_filters=512, out_filters=256, kernel_size=(1, 3), strides=1, padding="same")
        self.branch_b_right = Conv2d_bn(in_filters=512, out_filters=256, kernel_size=(3, 1), strides=1, padding="same")
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        x_1 = self.avg_block(x)
        x_2 = self.one_by_one_block(x)
        x_a = self.branch_a(x)
        x_3 = self.branch_a_left(x_a)
        x_4 = self.branch_a_right(x_a)
        x_b = self.branch_b(x)
        x_5 = self.branch_b_left(x_b)
        x_6 = self.branch_b_right(x_b)
        x = torch.cat([x_1, x_2, x_3, x_4, x_5, x_6], axis=1)
        return x

In [13]:
class Reduction_A(Module):

    def __init__(self, in_filters):
        super().__init__()
        self.max_pool = MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.central_block = Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=3, strides=2, padding="valid")
        self.right_block =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=192, out_filters=224, kernel_size=3, strides=1, padding="same"),
            Conv2d_bn(in_filters=224, out_filters=256, kernel_size=3, strides=2, padding="valid"),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        x_1 = self.max_pool(x)
        x_2 = self.central_block(x)
        x_3 = self.right_block(x)
        x = torch.cat([x_1, x_2, x_3], axis=1)
        return x

In [14]:
class Reduction_B(Module):

    def __init__(self, in_filters):
        super().__init__()
        self.max_pool = MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.central_block = Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=192, out_filters=192, kernel_size=3, strides=2, padding="valid"),
        )
        self.right_block =  Sequential(
            Conv2d_bn(in_filters=in_filters, out_filters=256, kernel_size=1, strides=1, padding="same"),
            Conv2d_bn(in_filters=256, out_filters=256, kernel_size=(1, 7), strides=1, padding="same"),
            Conv2d_bn(in_filters=256, out_filters=320, kernel_size=(7, 1), strides=1, padding="same"),
            Conv2d_bn(in_filters=320, out_filters=320, kernel_size=3, strides=2, padding="valid"),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        x_1 = self.max_pool(x)
        x_2 = self.central_block(x)
        x_3 = self.right_block(x)
        x = torch.cat([x_1, x_2, x_3], axis=1)
        return x

In [15]:
class InceptionV4(Module):

    def __init__(self):
        super().__init__()
        self.stem = StemBlock()
        self.inception_a = Sequential(
            A_block(384),
            A_block(384),
            A_block(384),
            A_block(384)
        )
        self.reduction_a = Reduction_A(384)
        self.inception_b = Sequential(
            B_block(1024),
            B_block(1024),
            B_block(1024),
            B_block(1024),
            B_block(1024),
            B_block(1024),
            B_block(1024)
        )
        self.reduction_b = Reduction_B(1024)
        self.inception_c = Sequential(
            C_block(1536),
            C_block(1536),
            C_block(1536)
        )
        self.drop = Dropout(0.2)
        #self.out = Linear(1536, 1)
        self.out = Linear(1536, 75)
        self.apply(self._init_weights)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_a(x)
        x = self.reduction_a(x)
        x = self.inception_b(x)
        x = self.reduction_b(x)
        x = self.inception_c(x)
        x = x.reshape(x.shape[0], -1, 1536).mean(axis=1)
        x = self.drop(x)
        y = self.out(x)
        return y

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

## Defining other Prerequisites

In [16]:
def top_k_accuracy(predictions, true, k):
    top_k_preds = predictions.topk(k, dim=1).indices
    correct = top_k_preds.eq(true.view(-1, 1).expand_as(top_k_preds))
    top_k_acc = correct.sum().float()/true.size(0)
    return top_k_acc

def normalize(data):
    return (data - data.min()) /  (data.max() - data.min())

def normalize_norm(data):
    return data/np.linalg.norm(data)

def normalize_range(data):
    top = 0.01
    btm = 0.99
    norm_01 = normalize(data)

    return norm_01*(top-btm) + btm


In [17]:
class FocalLoss(Module):

    def __init__(self,
                 alpha = None,          #tensor, Weights for each class
                 gamma = 0.):           #Const

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

        self.cross_entropy_loss = CrossEntropyLoss(reduction = 'none')

    def forward(self, x, y):

        ce = self.cross_entropy_loss(x, y)
        pt = torch.exp(-ce)
        focal_loss = self.alpha[y] * (1 - pt) ** self.gamma * ce

        focal_loss = focal_loss.mean()

        return focal_loss

In [18]:
classCount_train = np.array([118, 201, 161, 236, 83, 92, 296, 14, 36, 135, 49, 59, 248, 592, 76, 19, 224, 47, 60, 226, 465, 69, 44, 19, 14, 199, 47, 71, 1201, 319, 195, 643, 937, 10, 845, 127, 22, 34, 13, 13, 133, 57, 299, 431, 288, 490, 84, 226, 14, 253, 587, 208, 360, 22, 30, 16, 381, 16, 228, 168, 17, 120, 20, 469, 241, 22, 288, 208, 299, 47, 498, 360, 350, 120, 407])
classCount_val = np.array([48, 86, 82, 103, 44, 50, 135, 5, 22, 64, 26, 25, 108, 266, 32, 6, 115, 27, 37, 96, 228, 31, 20, 6, 4, 86, 25, 33, 570, 119, 99, 293, 439, 3, 405, 54, 11, 11, 4, 5, 70, 34, 134, 208, 128, 217, 45, 95, 3, 111, 300, 110, 145, 5, 11, 8, 195, 10, 104, 71, 7, 54, 7, 219, 121, 11, 142, 94, 130, 18, 252, 185, 156, 49, 197])
classCount_test = np.array([60, 107, 67, 122, 36, 32, 143, 7, 11, 58, 16, 29, 117, 282, 38, 11, 97, 16, 20, 117, 210, 32, 21, 9, 7, 96, 19, 36, 555, 164, 87, 321, 452, 7, 394, 64, 7, 19, 5, 5, 58, 21, 149, 196, 142, 233, 35, 115, 6, 129, 258, 80, 181, 16, 16, 6, 165, 4, 110, 89, 6, 52, 9, 212, 105, 10, 128, 97, 137, 24, 219, 160, 166, 61, 179])

classCountAll = classCount_train + classCount_val + classCount_test

invClassFrequency = sum(classCountAll) / (len(classCountAll)*classCountAll)
invClassFrequency = normalize_range(invClassFrequency)
print(sum(invClassFrequency), invClassFrequency)
weights = (torch.from_numpy(invClassFrequency))

59.55098894569825 [0.91102173 0.94832192 0.9347254  0.95561455 0.87721128 0.88487893
 0.96405707 0.23811528 0.71197795 0.92157353 0.78124691 0.82354389
 0.95670254 0.98115747 0.86308867 0.44933314 0.95315555 0.778833
 0.82952528 0.95346542 0.97660589 0.8487269  0.76591143 0.41702923
 0.20769991 0.94660983 0.78124691 0.85728534 0.99       0.96565905
 0.94660983 0.98277165 0.98768447 0.01       0.98647402 0.91780572
 0.50424978 0.68959345 0.0998636  0.13893473 0.92275247 0.82198179
 0.96453051 0.97482293 0.96306947 0.97746766 0.87795084 0.95315555
 0.13893473 0.95839816 0.9812332  0.94882622 0.96968034 0.53873233
 0.65165761 0.33949986 0.97181942 0.33949986 0.95377108 0.9382252
 0.33949986 0.91102173 0.44933314 0.97653291 0.95616554 0.53873233
 0.96306947 0.94895072 0.96357025 0.77636483 0.9780971  0.97045703
 0.96907994 0.91254308 0.97325054]


## Training the Network

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

weights = weights.to(device)

model = InceptionV4()
#opt = SGD(model.parameters(), lr=0.005)
opt = Adam(model.parameters(), lr=0.001)
#loss_fn = CrossEntropyLoss()
loss_fn = FocalLoss(alpha = weights, gamma = 2)

model.to(device)


epochs = 5
best_val = np.inf

if __name__ == "__main__":

    train_dataset = ImageDataset("/content/Car-make-classification/data/resized_images", "train", transforms)


    test_dataset = ImageDataset("/content/Car-make-classification/data/resized_images", "test", ToTensor())


    valid_dataset = ImageDataset("/content/Car-make-classification/data/resized_images", "validation", ToTensor())

    batch_size = 64
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())
    valid_dataloader = DataLoader(valid_dataset, batch_size, shuffle=False, num_workers=os.cpu_count())
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=os.cpu_count())

    for epoch in range(epochs):
        model.train()
        print(f"Epoch: {epoch+1}")
        iterator = tqdm(train_dataloader)
        for batch_x, batch_y in iterator:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            loss = loss_fn(y_pred, batch_y)

            opt.zero_grad()
            loss.backward()
            opt.step()
            iterator.set_description(f"Train loss: {loss.detach().cpu().numpy()}")

        model.eval()
        with torch.no_grad():
            predictions = []
            true = []
            for batch_x, batch_y in tqdm(valid_dataloader):
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)

                y_pred = model(batch_x)

                predictions.append(y_pred)
                true.append(batch_y)
            predictions = torch.cat(predictions, axis=0)
            true = torch.cat(true, axis=0)
            val_loss = loss_fn(predictions, true)

            val_acc = (predictions.argmax(dim=1) == true).float().mean()  # Use argmax for multi-class accuracy
            val_acc_top_5 = top_k_accuracy(predictions, true, 5)
            val_acc_top_10 = top_k_accuracy(predictions, true, 10)

            print(f"loss: {val_loss}, accuracy: {val_acc},\n top 5 acc: {val_acc_top_5}, top 10 acc: {val_acc_top_10}")

        if val_loss < best_val:
            print("Saved Model")
            torch.save(model.state_dict(), "model_FL.pt")
            best_val = val_loss

Epoch: 1


100%|██████████| 117/117 [00:06<00:00, 18.57it/s]


loss: 3.2933170054106715, accuracy: 0.07604765892028809,
 top 5 acc: 0.2857142686843872, top 10 acc: 0.439148485660553
Saved Model
Epoch: 2


100%|██████████| 117/117 [00:06<00:00, 19.18it/s]


loss: 3.240254949387132, accuracy: 0.07591377198696136,
 top 5 acc: 0.2945508062839508, top 10 acc: 0.4467800259590149
Saved Model
Epoch: 3


...

loss: 3.1853808237089085, accuracy: 0.08863301575183868,
 top 5 acc: 0.3033873438835144, top 10 acc: 0.47235238552093506
Saved Model
Epoch: 4


100%|██████████| 117/117 [00:06<00:00, 18.72it/s]


loss: 3.081217702013372, accuracy: 0.09921006858348846,
 top 5 acc: 0.33190521597862244, top 10 acc: 0.4944436848163605
Saved Model
Epoch: 5


100%|██████████| 117/117 [00:06<00:00, 19.22it/s]


loss: 3.0564423345726275, accuracy: 0.09331905096769333,
 top 5 acc: 0.33324405550956726, top 10 acc: 0.5015397071838379
Saved Model
